In [1]:
import logging
import os


from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import pandas as pd
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
load_dotenv()


USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
llm=ChatGroq(model="llama-3.2-11b-text-preview",api_key=os.environ['GROQ_API_KEY'])

In [3]:
url=['https://www.infosys.com']
content=WebBaseLoader(url).load()
content

[Document(metadata={'source': 'https://www.infosys.com', 'title': 'Infosys - Consulting | IT Services | Digital Transformation', 'description': 'Infosys is a global leader in next-generation digital services and consulting.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nInfosys - Consulting | IT Services | Digital Transformation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n\n\n Navigate your next \n Infosys Knowledge Institute \n Investors \n Careers \n\n\n\n\n\n\n\nsearch\n\n\n\ncross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\nHit enter to search or ESC to close\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   \n\n Navigate your next   \n Industries   \n Services   \n Platforms   \n Infosys Knowledge Institute\r\n                  \n About Us   \n Investors   \n Careers\r\n                  \n Newsroom   \n Contact Us   \

In [4]:
document=RecursiveCharacterTextSplitter(chunk_size=4000,chunk_overlap=200).split_documents(content)
document

[Document(metadata={'source': 'https://www.infosys.com', 'title': 'Infosys - Consulting | IT Services | Digital Transformation', 'description': 'Infosys is a global leader in next-generation digital services and consulting.', 'language': 'en'}, page_content="Infosys - Consulting | IT Services | Digital Transformation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n \n\n\n\n Navigate your next \n Infosys Knowledge Institute \n Investors \n Careers \n\n\n\n\n\n\n\nsearch\n\n\n\ncross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\nHit enter to search or ESC to close\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n×\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   \n\n Navigate your next   \n Industries   \n Services   \n Platforms   \n Infosys Knowledge Institute\r\n                  \n About Us   \n Investors   \n Careers\r\n                  \n Newsroom   \n Contact Us   \n \n\n\n\n \n \n\n\n\n \n \n\n\n\n \n \n\n\n\n

In [5]:
class EmailStructure(BaseModel):
    subject: str = Field(description="The subject line of the email, should be concise, attention-grabbing, and relevant to the recipient.")
    body: str = Field(
        description="The main content of the email, including an engaging introduction, personalized messaging, a clear call to action (CTA), and formatting that enhances readability. "
                    "Consider including dynamic content (such as the recipient's name, relevant offers, or interests), bullet points for clarity, and emotional language to evoke a response. "
                    "The tone should be friendly but professional, maintaining a balance between informative and persuasive elements to increase engagement rates."
                    "No placeholder should be there, it must be complete",
    )
parser = JsonOutputParser(pydantic_object=EmailStructure)

In [6]:
custom_prompt="""
        Generate a cold email with the following components:
        1. A compelling subject line that grabs attention
        2. A friendly greeting with talking abut there problem 
        3. An introduction as "Akshay Kumar BM "freelancer who specializes in the specified purpose
        4. A clear value proposition based on the recipient's website content
        5. An offer of a free trial or initial consultation
        6. A call to action (CTA) to engage the recipient
        7. A polite sign-off
        contact information : akshaykumarbedre.bm@gmail.com
        """
email_prompt = PromptTemplate(
            template="""
            Purpose: {purpose}\n\n
            Use the information below to craft the email:\n\n
            {text}\n\n
            """+custom_prompt+"""\n\n
            Also, provide a suitable subject line for this cold email.
            {format_instructions}
            COLD EMAIL:
            """,
            input_variables=["text", "purpose"],
            partial_variables={"format_instructions": parser.get_format_instructions()}
        )

refine_prompt = PromptTemplate(
            template="""
            Purpose: {purpose}\n\n

            We have provided a draft cold email: {existing_answer}
            Refine the email based on additional context below to make it more compelling or leave it as is if not needed.
            ------------
            {text}
            ------------
            Refine the cold email in English based on the new context. If the new context is not useful, return the original cold email.
            {format_instructions}
            """,
            input_variables=["existing_answer", "text","purpose"],
            partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [7]:
chain=load_summarize_chain(
            llm=llm,
            chain_type="refine",
            question_prompt=email_prompt,
            refine_prompt=refine_prompt,
            return_intermediate_steps=False,
        )


In [8]:
purpose="To offer automated personalized cold email services"

In [9]:
result = chain.invoke({
                "input_documents": document,
                "purpose": purpose
            })

In [11]:
result["output_text"]

'Based on the new context, the refined cold email in JSON format is:\n\n```json\n{\n  "properties": {\n    "subject": "Unlock the Power of Personalized Cold Email Services for Infosys\'s Digital Transformation",\n    "body": "Hi there,\\n\\nI came across the latest Infosys announcements on navigating your next-gen digital transformation journey, focusing on core capabilities, operating models, talent transformations, and more. I was impressed by the comprehensive approach you\'re taking to drive business growth and innovation.\\n\\nAs a freelancer who specializes in automated personalized cold email services, I understand the challenges of effectively reaching potential clients in the competitive tennis and tech industries. My services can help you streamline your email marketing efforts, increase engagement rates, and ultimately drive more conversions.\\n\\nConsidering Infosys\'s commitment to amplifying human potential and creating the next opportunity for people, businesses, and com